In [3]:
import os
import sys
import torch
import random
import argparse
import numpy as np
import scipy
from matplotlib import pyplot
import numpy

sys.path.append('../../timeseries/')
sys.path.append('../../timeseries/experiments_scripts/')
sys.path.append('../../src/')

import factors
import state_models
import observation_models
from ssm_em import StateSpaceEM
from nonlinear_ssm import NonLinearStateSpace_EM

from scipy.stats import norm
from scipy.stats import zscore
from ssm_em import StateSpaceEM, load_model
from sklearn.covariance import EmpiricalCovariance
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.gaussian_process.kernels import ExpSineSquared, RBF

from exp_utils import *
import ssm
import statsmodels.api as sm

'''
sys.path.append('../../timeseries/kalman-jax-master')
from jax.experimental import optimizers
#from sde_gp import SDEGP
import approximate_inference as approx_inf
import priors
import likelihoods
from utils import softplus_list, plot
'''


class PredictiveDensity:
    def __init__(self, mu, sigma):
        if mu.ndim == 1:
            self.mu = np.array([mu]).T
        else:
            self.mu = np.array(mu)
        if sigma.ndim == 1:
            self.Sigma = np.array([sigma]).T
        else:    
            self.Sigma = np.array(sigma)
            
def reset_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
def train_linear_SSM(x_tr, dz, init_w_pca=False, smooth_window=20):

    dx = x_tr.shape[1]
    sm = state_models.LinearStateModel(dz)
    om = observation_models.LinearObservationModel(dx, dz, noise_x=1.)
    
    if init_w_pca == 1:
        om.pca_init(x_tr, smooth_window=smooth_window)
        
    ssm_em_lin = StateSpaceEM(x_tr, observation_model=om, state_model=sm)
    ssm_em_lin.run()
    
    return ssm_em_lin


def train_linear_hsk_SSM(x_tr, dz, du, init_w_pca=False, smooth_window=20):

    dx = x_tr.shape[1]
    sm_hs = state_models.LinearStateModel(dz)
    om_hs = observation_models.HCCovObservationModel(dx, dz, du)
    if init_w_pca == 1:
        om_hs.pca_init(x_tr, smooth_window=20)
    hs_model = StateSpaceEM(x_tr, observation_model=om_hs, state_model=sm_hs, conv_crit=1e-4)
    hs_model.run()
    
    return hs_model

def train_nonlinear_SSM(x_tr, **kwargs):
    '''
    to be updated; currently doesn't run
    LSEMStateModel -> sm_hs = state_models.LSEMStateModel(args.dz,args.dk# + param for basis func)
    '''
    nonlin_model = NonLinearStateSpace_EM(x_tr,args.dz, args.dk)
    nonlin_model.run()
    
    return nonlin_model

class HMM_class:
    
    def __init__(self, x_tr, K, obs_model='gaussian'):
        self.x_tr = x_tr
        self.D = x_tr.shape[1]
        self.K = K
        self.obs_model = obs_model
        self.model = self._train()
        
    def _train(self):
        model = ssm.HMM(self.K, self.D, observations=self.obs_model)
        model.fit(self.x_tr, method="em")
        return model

    def compute_predictive_log_likelihood(self, x_te):
        return self.model.log_likelihood(x_te)
    
    def compute_predictive_density(self, x_te):
        mask = np.logical_not(np.isnan(x_te))
        x_te_not_nan = np.zeros(x_te.shape)
        x_te_not_nan[mask] = x_te[mask]
        states = self.model.filter(x_te_not_nan, mask=mask)
        if self.obs_model == 'gaussian' or  self.obs_model == 'studentst':
            mean_te = np.dot(states, self.model.observations.mus)
        elif self.obs_model == 'ar':
            mean_te = np.sum(states[:,:,None] * (np.sum(self.model.observations.As[None] * x_te[:,None, None], axis=3) + self.model.observations.bs), axis=1)
        std_te = np.dot(states, np.sqrt(self.model.observations.Sigmas.diagonal(axis1=1, axis2=2)))
        print(mean_te.shape, std_te.shape)
        return PredictiveDensity(mean_te, std_te ** 2)

def train_HMM(x_tr, num_states, **kwargs):
    return HMM_class(x_tr, num_states)


class ARIMAX:
    
    def __init__(self, x_tr, p, q):
        self.x_tr = x_tr
        self.p = p
        self.q = q
        self._train()
        
    def _train(self):
        if x_tr.shape[1] == 1:
            self.mod = sm.tsa.statespace.SARIMAX(x_tr, trend='c', order=(self.p,0,self.q))
            self.fit_res = self.mod.fit(disp=False)
        else:
            self.mod = sm.tsa.VARMAX(x_tr, trend='c', order=(self.p,self.q))
            self.fit_res = self.mod.fit(disp=False, max_iter=1000)
            
    def compute_predictive_density(self, x_te):
        mod_te = self.mod.clone(x_te)
        res = mod_te.filter(self.fit_res.params)
        predict = res.get_prediction()
        predict_ci = predict.conf_int(alpha=1.-.68)
        mu = predict.predicted_mean
        if x_te.shape[1] == 1:
            std = predict.predicted_mean - predict_ci[:,0]
        else:
            std = predict.predicted_mean - predict_ci[:,:x_te.shape[1]]
        return PredictiveDensity(mu, std ** 2)
            
    def compute_predictive_log_likelihood(self, x_te):  
        mod_te = self.mod.clone(x_te)
        return mod_te.loglike(self.fit_res.params)
    
def train_arimax(x_tr, p, q,**kwargs):
    
    arimax_model = ARIMAX(x_tr, p, q)
    return arimax_model

In [57]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_energy_e1(train_ratio=.5)
dz = 3
ssm_em_lin = train_linear_SSM(x_tr, dz, init_w_pca=True)

Iteration 2 - llk=-2485.9
Iteration 4 - llk=-1633.0
Iteration 6 - llk=-1227.0
Iteration 8 - llk=-1017.7
Iteration 10 - llk=-937.7
Iteration 12 - llk=-883.5
Iteration 14 - llk=-828.5
Iteration 16 - llk=-778.8
Iteration 18 - llk=-741.5
Iteration 20 - llk=-716.8
Iteration 22 - llk=-700.7
Iteration 24 - llk=-689.7
Iteration 26 - llk=-681.5
Iteration 28 - llk=-675.2
Iteration 30 - llk=-669.9
Iteration 32 - llk=-665.2
Iteration 34 - llk=-661.0
Iteration 36 - llk=-657.2
Iteration 38 - llk=-653.6
Iteration 40 - llk=-650.2
Iteration 42 - llk=-647.0
Iteration 44 - llk=-644.0
Iteration 46 - llk=-641.2
Iteration 48 - llk=-638.5
Iteration 50 - llk=-636.0
Iteration 52 - llk=-633.5
Iteration 54 - llk=-631.1
Iteration 56 - llk=-628.8
Iteration 58 - llk=-626.6
Iteration 60 - llk=-624.4
Iteration 62 - llk=-622.3
Iteration 64 - llk=-620.3
Iteration 66 - llk=-618.4
Iteration 68 - llk=-616.6
Iteration 70 - llk=-614.9
Iteration 72 - llk=-613.3
Iteration 74 - llk=-611.9
Iteration 76 - llk=-610.5
Iteration 78

In [58]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_energy_e1(train_ratio=.5)
dz = 3
du = 1
ssm_em_hsk = train_linear_hsk_SSM(x_tr, dz, du, init_w_pca=True)

Iteration 2 - llk=-1198.5
Iteration 4 - llk=-683.0
Iteration 6 - llk=-543.5
Iteration 8 - llk=-465.6
Iteration 10 - llk=-427.2
Iteration 12 - llk=-409.2
Iteration 14 - llk=-402.6
Iteration 16 - llk=-399.5
Iteration 18 - llk=-398.6
Iteration 20 - llk=-398.3
Iteration 22 - llk=-398.1
EM did converge.


In [60]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_energy_e1(train_ratio=.5)
K = 6
hmm = train_HMM(x_tr, K)

  0%|          | 0/100 [00:00<?, ?it/s]

In [61]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_energy_e1(train_ratio=.5)
p, q = 3, 0
arimax = train_arimax(x_tr, p, q)

/opt/conda/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [62]:
T_start = 10
T_stop = 903
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_energy_e1(train_ratio=.5)
lin_pred_density = ssm_em_lin.compute_predictive_density(x_te_original)
lb_lin, ub_lin = lin_pred_density.mu[T_start:T_stop] - 1.68 * numpy.sqrt(lin_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2)), lin_pred_density.mu[T_start:T_stop] + 1.68 * numpy.sqrt(lin_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2))
lb_lin, ub_lin = s_tr_x.inverse_transform(lb_lin), s_tr_x.inverse_transform(ub_lin)
lb_lin[:,[0,2,3]] = numpy.logaddexp(1, lb_lin[:,[0,2,3]])
ub_lin[:,[0,2,3]] = numpy.logaddexp(1, ub_lin[:,[0,2,3]])
hsk_pred_density = ssm_em_hsk.compute_predictive_density(x_te_original)
lb_hsk, ub_hsk = hsk_pred_density.mu[T_start:T_stop] - 1.68 * numpy.sqrt(hsk_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2)), hsk_pred_density.mu[T_start:T_stop] + 1.68 * numpy.sqrt(hsk_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2))
lb_hsk, ub_hsk = s_tr_x.inverse_transform(lb_hsk), s_tr_x.inverse_transform(ub_hsk)
lb_hsk[:,[0,2,3]] = numpy.logaddexp(1, lb_hsk[:,[0,2,3]])
ub_hsk[:,[0,2,3]] = numpy.logaddexp(1, ub_hsk[:,[0,2,3]])
hmm_pred_density = hmm.compute_predictive_density(x_te_original)
lb_hmm, ub_hmm = hmm_pred_density.mu[T_start:T_stop] - 1.68 * numpy.sqrt(hmm_pred_density.Sigma[T_start:T_stop]), hmm_pred_density.mu[T_start:T_stop] + 1.68 * numpy.sqrt(hmm_pred_density.Sigma[T_start:T_stop])
lb_hmm, ub_hmm = s_tr_x.inverse_transform(lb_hmm), s_tr_x.inverse_transform(ub_hmm)
lb_hmm[:,[0,2,3]] = numpy.logaddexp(1, lb_hmm[:,[0,2,3]])
ub_hmm[:,[0,2,3]] = numpy.logaddexp(1, ub_hmm[:,[0,2,3]])
arimax_pred_density = arimax.compute_predictive_density(x_te_original)
lb_arimax, ub_arimax = arimax_pred_density.mu[T_start:T_stop] - 1.68 * numpy.sqrt(arimax_pred_density.Sigma[T_start:T_stop]), arimax_pred_density.mu[T_start:T_stop] + 1.68 * numpy.sqrt(arimax_pred_density.Sigma[T_start:T_stop])
lb_arimax, ub_arimax = s_tr_x.inverse_transform(lb_arimax), s_tr_x.inverse_transform(ub_arimax)
lb_arimax[:,[0,2,3]] = numpy.logaddexp(1, lb_arimax[:,[0,2,3]])
ub_arimax[:,[0,2,3]] = numpy.logaddexp(1, ub_arimax[:,[0,2,3]])

(913, 4) (913, 4)


In [63]:
from timeseries_data import load_opsd_data
raw_data = load_opsd_data().iloc[-x_te_original.shape[0]:]
date = raw_data.index
tick_idx = numpy.where(date.is_month_start)[0]
tick_labels = []
for i, idx in enumerate(tick_idx):
    label = date[idx].month_name()[:3] + str(date[idx].year)[-2:]
    tick_labels.append(label)
tick_labels

['Aug17',
 'Sep17',
 'Oct17',
 'Nov17',
 'Dec17',
 'Jan18',
 'Feb18',
 'Mar18',
 'Apr18',
 'May18',
 'Jun18',
 'Jul18',
 'Aug18',
 'Sep18',
 'Oct18',
 'Nov18',
 'Dec18',
 'Jan19',
 'Feb19',
 'Mar19',
 'Apr19',
 'May19',
 'Jun19',
 'Jul19',
 'Aug19',
 'Sep19',
 'Oct19',
 'Nov19',
 'Dec19']

In [50]:
raw_data.columns

Index(['DE_solar_generation_actual', 'DE_temperature',
       'DE_radiation_direct_horizontal', 'DE_radiation_diffuse_horizontal'],
      dtype='object')

In [71]:
pyplot.rcdefaults()
pyplot.rc('font', size = 14)
fig = pyplot.figure(figsize=(16,8))
gs = fig.add_gridspec(4, 6, hspace=0)

x_te_plot = s_tr_x.inverse_transform(x_te_original)
x_te_plot[:,[0,2,3]] = numpy.logaddexp(1, x_te_plot[:,[0,2,3]])

for dx in range(ssm_em_lin.Dx):
    ax1 = fig.add_subplot(gs[dx,:4])
    ax1.plot(range(T_start,T_stop), x_te_plot[T_start:T_stop, dx], 'k.', ms=1)
    ax1.fill_between(range(T_start,T_stop), lb_arimax[:,dx], ub_arimax[:,dx], color='C1', facecolor='none', alpha=1, lw=1)
    ax1.fill_between(range(T_start,T_stop), lb_lin[:,dx], ub_lin[:,dx], color='C0', alpha=.4, lw=0)
    ax1.fill_between(range(T_start,T_stop), lb_hmm[:,dx], ub_hmm[:,dx], color='C2', facecolor='none', alpha=.8, lw=1)
    ax1.fill_between(range(T_start,T_stop), lb_hsk[:,dx], ub_hsk[:,dx], color='C3', alpha=.4, lw=0)
    ax1.set_xlim([T_start,T_stop])
    ax1.set_xticks([])
    #ax1.set_yticks([])
    if dx >= 2:
        ax1.set_ylabel(raw_data.columns[dx][3:13] + ' \n ' + raw_data.columns[dx][13:], fontsize=10)
    else:
        ax1.set_ylabel(raw_data.columns[dx][3:], fontsize=10)
    
    
    ax2 = fig.add_subplot(gs[dx,4])
    hsk_cdf = numpy.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - hsk_pred_density.mu[:,dx]) / numpy.sqrt(hsk_pred_density.Sigma[:,dx,dx])))
    lin_cdf = numpy.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - lin_pred_density.mu[:,dx]) / numpy.sqrt(lin_pred_density.Sigma[:,dx,dx])))
    hmm_cdf = numpy.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - hmm_pred_density.mu[:,dx]) / numpy.sqrt(hmm_pred_density.Sigma[:,dx])))
    arimax_cdf = numpy.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - arimax_pred_density.mu[:,dx]) / numpy.sqrt(arimax_pred_density.Sigma[:,dx])))
    ax2.plot(numpy.linspace(0,1,x_te_original.shape[0]), lin_cdf, color='C0', alpha=.7, lw=2)
    ax2.plot(numpy.linspace(0,1,x_te_original.shape[0]), hmm_cdf, color='C2', alpha=.7, lw=2)
    ax2.plot(numpy.linspace(0,1,x_te_original.shape[0]), hsk_cdf, color='C3', alpha=1)
    ax2.plot(numpy.linspace(0,1,x_te_original.shape[0]), arimax_cdf, color='C1', alpha=.7)
    ax2.plot([0,1], [0,1], 'k--', lw=1., alpha=.5)
    ax2.set_xlim([0,1])
    ax2.set_ylim([0,1])
    ax2.set_xticks([])
    ax2.set_yticks([0,1])
    if dx == 0:
        ax2.set_yticklabels(labels=['','1'], fontsize=12)
    else:
        ax2.set_yticklabels(labels=['','0\n 1'], fontsize=12)
    
    ax3 = fig.add_subplot(gs[dx,5])
    ax3.plot(numpy.linspace(0,1,x_te_original.shape[0]), 100 * (lin_cdf - numpy.linspace(0,1,x_te_original.shape[0])), color='C0', alpha=.7, lw=2)
    ax3.plot(numpy.linspace(0,1,x_te_original.shape[0]), 100 * (hmm_cdf - numpy.linspace(0,1,x_te_original.shape[0])), color='C2', alpha=.7, lw=2)
    ax3.plot(numpy.linspace(0,1,x_te_original.shape[0]), 100 * (arimax_cdf - numpy.linspace(0,1,x_te_original.shape[0])), color='C1', alpha=.7, lw=2)
    ax3.plot(numpy.linspace(0,1,x_te_original.shape[0]), 100 * (hsk_cdf - numpy.linspace(0,1,x_te_original.shape[0])), color='C3')
    ax3.plot([0,1], [0,0], 'k--', lw=1., alpha=.5)
    ax3.set_xlim([0,1])
    #ax2.set_ylim([0,1])
    ax3.set_xticks([])
    ax3.set_yticks([-10,10])
    ax3.set_ylim([-15,15])
    if dx == 2:
        ax2.set_ylabel('Model quantile')
        ax3.set_ylabel('$\Delta$ quantile [%]')
    if dx == 0:
        ax1.set_title('(a)', loc='left', fontweight='bold', x=-.1)
        ax2.set_title('(b)', loc='left', fontweight='bold', x=-.5)
        ax3.set_title('(c)', loc='left', fontweight='bold', x=-.5)

#ax1.set_xlabel('Timesteps')
ax1.set_xticks(tick_idx[::2])
ax1.set_xticklabels(tick_labels[::2], rotation=45, fontsize=10)
ax2.set_xticks([0,1])
ax3.set_xticks([0,1])
ax2.set_xticks([0,1])
ax2.set_xlabel('Data quantile')
ax2.set_yticklabels(labels=['0','0\n 1'], fontsize=12)
ax3.set_xlabel('Data quantile')
pyplot.tight_layout()
fig.savefig('../../figs/aistats22/energy.pdf')
fig.savefig('../../figs/aistats22/energy.png', dpi=600)